# writers

The Midgard **writers** module can be used for writing files. For example following file formats are available via Midgard:

- Bernese station information files (*.ABB, *.CLU, *.CRD, *.STA, *.VEL)
- GipsyX site information file
- GAMIT station information file
- ...

All available writers can be shown with `names` method:

In [ ]:
# Import writers package
from midgard import writers

# List all available writers
writers.names()

## Use of station information file writers 

Following station information file writers exists:

| Writer name        | Description |
|:------------------ | :-----------|
| bernese_abb        | Bernese station abbreviation table in *.ABB format |
| bernese_clu        | Bernese station list file in *.CLU format |
| bernese_crd        | Bernese station coordinate file in *.CRD format | 
| bernese_sta        | Bernese station information file in *.STA format |
| bernese_vel        | Bernese station velocity file in *.VEL format |
| gamit_apr_eq       | GAMIT station coordinates files in *.apr and *.eq format |
| gamit_station_info | GAMIT station information file |
| gipsyx_site_info   | GipsyX station information file |

As a first step the input data has to be generated for the writers. That means station information has to be collected from one or more of the following sources:

| Source             | Parser name | Description |
|:------------------ | :-----------| :-----------|
| SINEX              | sinex_site  | Station information given in SINEX format files, which can inlude information about antenna, receiver, eccentricity and station coordinates (like https://files.igs.org/pub/station/general/igs.snx) | 
| SSC                | ssc_site    |Station coordinate and velocity is given in SSC format files (like https://itrf.ign.fr/docs/solutions/itrf2014/ITRF2014_GNSS.SSC.txt) |
| M3G                | m3g         | M3G application programming interface (API) from https://gnss-metadata.eu/site/api-docs. The M3G is a metadata management and distribution system for multiple GNSS networks (https://gnss-metadata.eu), which is accessable via M3G application programming interface (API).  |

Midgard **parsers** can be used to read SSC/SINEX files and the **site_info** module to access the M3G API. In the following a SSC and a SINEX file are read as **source_data**. These **source_data** are applied to initialize a Midgard **site_info** object. Midgard **site_info** class provides a data structure to save relevant station information. 

In [ ]:
# Standard library import
from datetime import datetime
from pprint import pprint # Used to print variables more pretty

# Import Midgard parsers
from midgard import parsers

# Import SiteCoord class
from midgard.site_info.site_info import SiteInfo

# Read station information (antenna, receiver, eccentricity) from SINEX file
p_snx = parsers.parse_file(
            parser_name="sinex_site", 
            file_path="../tests/parsers/example_files/sinex_site",
)

# Read station coordinates/velocities from SSC file
p_ssc = parsers.parse_file(
            parser_name="ssc_site", 
            file_path="../tests/parsers/example_files/ssc_site",
)

# Get station dictionary of SiteInfo instance for SINEX source
site_info = SiteInfo.get(
            source="snx", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p_snx.as_dict(),
            stations=p_snx.as_dict().keys(), # List with station identifiers
            source_path=p_snx.file_path,
)

# Get station dictionary of SiteInfo instance for SSC source
site_info_ssc = SiteInfo.get(
            source="ssc", # File type specifier: snx = SINEX; ssc = SSC
            source_data=p_ssc.as_dict(),
            stations=p_ssc.as_dict().keys(), # List with station identifiers
            source_path=p_ssc.file_path,
)

# Show site_info dictionary
pprint(site_info)

The SINEX files like **igs.snx** or **euref.snx** include only approximated station coordinates. Therefore we read in addition the SSC file with precise coordinates and velocities. The SINEX **site_info** dictionary will now be updated with the **site_coord** objects of the SSC **site_info_ssc** dictionary.

In [ ]:
# Update SINEX source with station coordinates/velocities from SSC source
for sta in site_info.keys():
    site_info[sta]["site_coord"] = site_info_ssc[sta]["site_coord"]
    
# Show site_info dictionary
pprint(site_info)

The last step is to use the **site_info** dictionary for writing the station information in different formats with the writers like **bernese_crd**, **bernese_sta**, **gamit_station_info** or **gipsyx_site_info**.

In [ ]:
# Import writer
from midgard.writers.bernese_crd import bernese_crd
from midgard.writers.bernese_sta import bernese_sta
from midgard.writers.bernese_vel import bernese_vel
from midgard.writers.gipsyx_site_info import gipsyx_site_info

# Use bernese_crd writer
bernese_crd(
    file_path="./examples/writers/BERNESE.CRD",
    site_info=site_info,
    datum="IGb14", # Reference system of station coordinates
    epoch=datetime(2010,1,1), # Reference epoch of station coordinates
    agency="NMA",
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.CRD', language='python')

In [ ]:
# Use bernese_vel writer
bernese_vel(
    file_path="./examples/writers/BERNESE.VEL",
    site_info=site_info,
    datum="IGb14", # Reference system of station velocities
    agency="NMA",
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.VEL', language='python')

In [ ]:
# Use bernese_sta writer
bernese_sta(
    file_path="./examples/writers/BERNESE.STA",
    site_info=site_info,
    agency="NMA", # Agency which uses this file for processing
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/BERNESE.STA', language='python')

In [ ]:
# Use gipsyx_site_info writer
gipsyx_site_info(
    file_path="./examples/writers/gipsyx_site_info",
    site_info=site_info,
)

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/gipsyx_site_info', language='python')

## Use of SINEX timeseries writer sinex_tms

An example is shown, how to use the SINEX timeseries writer **sinex_tms** module. As a first step the input data has to be generated. In this example several daily station coordinate solutions in SINEX format are read to generate the needed Midgard dataset as input for the **sinex_tms** writer.

In [2]:
# Standard library import
from datetime import datetime, timedelta

# Import Midgard parsers
import midgard
from midgard.data.dataset import Dataset
from midgard import parsers

# Initialize Midgard dataset
dset = Dataset()

# Define time period
date_from = datetime(2023, 6, 9)
date_to = datetime(2023, 6, 11)

# Loop over time period
date = date_from
while date <= date_to:
    
    # Read SINEX file with station coordinate solutions
    yyddd = date.strftime("%y%j")
    p = parsers.parse_file(
                parser_name="sinex_site", 
                file_path=f"./examples/parsers/F1_{yyddd}0.SNX",
    )
    
    if date_from == date:
        # Initialize first dataset entry
        dset = p.as_dataset()
    else:
        # Add following daily SINEX file solution 
        dset.extend(p.as_dataset())
    
    # Increase date counter
    date += timedelta(days=1)

The next step is to use the generated dataset **dset** for writing SINEX timeseries files for each station.

In [3]:
# Import writer
from midgard.writers.sinex_tms import sinex_tms

# Loop over stations
for sta in dset.unique("station"):

    # Use sinex_tms writer
    sinex_tms(
        dset=dset,
        station=sta,
        file_path=f"./examples/writers/{sta}.tms",
        contact="dummy@kartverket.no",
        data_agency="NMA",
        file_agency="NMA",
        input_="Daily GNSS Solutions processed with Bernese 5.4",
        organization="Norwegian Mapping Authority",
        software=f"Midgard {midgard.__version__}",
        version="001",
    )

# Display output file
from IPython.display import Code
Code(filename='./examples/writers/brux.tms', language='python')

%=TMS 1.0 NMA 2024:042:80484 NMA 2023:160:43185 2023:162:43185 P BRUX
+FILE/REFERENCE
*INFO_TYPE________: INFO___________________________________________________________________________________
 DESCRIPTION        Norwegian Mapping Authority
 OUTPUT             GNSS station coordinate timeseries solution                 
 CONTACT            dummy@kartverket.no                                         
 SOFTWARE           Midgard 1.3.0                                               
 INPUT              Daily GNSS Solutions processed with Bernese 5.4             
 VERSION NUMBER     001                                                         
-FILE/REFERENCE
+TIMESERIES/COLUMNS
*__COL __NAME______________ __UNIT______________ __DESCRIPTION__________________________________________________
     1 YYYY-MM-DD                                Date in format year, month and day (e.g. 2023-06-01)
     2 YEAR                 y                    Date as decimal year (2023.4137)
     3 X                    m                    X-coordinate of geocentric site coordinates
     4 Y                    m                    Y-coordinate of geocentric site coordinates
     5 Z                    m                    Z-coordinate of geocentric site coordinates
     6 SIG_X                m                    Standard deviation of geocentric X-coordinate
     7 SIG_Y                m                    Standard deviation of geocentric Y-coordinate
     8 SIG_Z                m                    Standard deviation of geocentric Z-coordinate
-TIMESERIES/COLUMNS
+TIMESERIES/DATA
* _YYYY-MM-DD _YEAR______ _X___________ _Y___________ _Z___________ _SIG_X___ _SIG_Y___ _SIG_Z___
 2023-06-09    2023.43699  4027881.3340   306998.8067  4919499.0515    0.0007    0.0003    0.0008
 2023-06-10    2023.43973  4027881.3328   306998.8066  4919499.0520    0.0007    0.0003    0.0009
 2023-06-11    2023.44247  4027881.3304   306998.8074  4919499.0505    0.0008    0.0003    0.0010
-TIMESERIES/DATA